In [17]:
import json
import logging
import os
import sys
import time
import psycopg2 as pg
from urllib.parse import urljoin
import tldextract
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import csv
import pickle

In [2]:
sources = ['https://mediabiasfactcheck.com/left/', 'https://mediabiasfactcheck.com/leftcenter/', 'https://mediabiasfactcheck.com/center/', 'https://mediabiasfactcheck.com/right-center/', 'https://mediabiasfactcheck.com/right/']
sources += ['https://mediabiasfactcheck.com/conspiracy/', 'https://mediabiasfactcheck.com/fake-news/', 'https://mediabiasfactcheck.com/satire/']
labels = ['L', 'LC', 'C', 'RC', 'R', 'X', 'F', 'S']
data = {}

In [10]:
for s, l in zip(sources, labels):
    print(s)
    source = urlopen(s)
    source = BeautifulSoup(source, 'lxml')
    for p in source.findAll('p'):
        if p.has_attr('style') and (p['style'] in ['text-align: center;', 'text-align:center;']):
            for tag in p.findAll('a', href=True):
                name = tag.text.replace("\n","")
                print(name)
                if name not in data:
                    plink = tag['href']
                    try:
                        page = BeautifulSoup(urlopen(tag['href']), 'lxml')
                        data[name] = {'fact':None,'url':None,'bias':None,'flag':None}

                        for p2 in page.findAll('p'):
                            if "Factual Reporting" in p2.text:
                                rank = p2.find('strong')
                                if rank == None:
                                    rank = p2.find('b')
                                if rank != None:
                                    data[name]['fact'] = rank.text
                                break
                        for p2 in page.findAll('p'):
                            if "Source:" in p2.text:
                                slink = p2.find('a')
                                if slink != None:
                                    data[name]['url'] = slink['href']
                                break
                        if l in labels[:5]:
                            data[name]['bias'] = l
                        else:
                            data[name]['flag'] = l        
                    except Exception as e:
                        print("oops")

https://mediabiasfactcheck.com/left/
Addicting Info
Advocate
All That’s Fab
Alternet
Amandla
AmericaBlog
American Bridge 21st Century
American News X
Backed by Fact
Bay Area Reporter
Bitch Media
Black Agenda Report
Black Lives Matter
Blue Dot Daily
Blue Nation Review
Boing Boing
Breitbart Unmasked
Bust Magazine
Canadian Dimension
Carbonated TV
Center For American Progress
Center for Media and Democracy (PR Watch)
Change.org
Chicago Reader
Chronicles of Democracy
CNN
Common Dreams
Cosmopolitan
CounterCurrents.org
CounterPunch
CrimethInc
Crooks and Liars
Current Affairs Magazine
Daily Beast
Daily Dot
Daily Kos
Daily Mirror
Daily News Bin
Dallas Voice
Dead State
DeathandTaxes
Democracy Now
Democratic Underground
DeSmogBlog
Die Hard Democrat
Dissent Magazine
Drudge Retort
Earth First! Journal
Economy in Crisis
Electronic Intifada
Elisabeth Parker
Elite Daily
Esquire Magazine
Everyday Feminism
Evonomics
Fifth Estate
Filming Cops
Forward Progressives
Gizmodo
Good Magazine
GOPOPCALYPSE
GQ Mag

9 News (Australia)
American Action Forum
American Council on Science and Health
American Foreign Policy Council
Anti-War
Arab News
Arizona Republic
Arkansas Democrat-Gazette
Boston Herald
Breaking Defense
Calgary Sun
California Political Review
Canadian Taxpayers Federation
Cato Institute
Center for a Stateless Society
Center for the National Interest
Christianity Today
Cincinnati Enquirer
Colorado Springs Gazette
Columbus Dispatch
Communities Digital News
Competitive Enterprise Institute
Council on Foreign Relations
Daily Press
Daily Telegraph
Dallas Morning News
Des Moines Register
Deseret News
Desert Sun
Detroit News
Edmonton Sun
Financial Post
Florida Times Union
Forbes
Foreign Policy Initiative
Foreign Policy Research Institute
Fortune Magazine
Foundation for Economic Education
Franklin Center for Government and Public Integrity
Freedom House
Grand Rapids Press
Group of Thirty
Gulf News
Herald-Sun (Australia)
High Point Enterprise
Hindu Business Line
Honest Reporting
Hoover Instit

100 Percent Fed Up
247 News Media
70 News
AbcNews.com.co
Age of Shitlords
Alabama Observer
Allen West Republic
All News Pipeline
Alternative Media Syndicate
American College of Pediatricians
American Fans
American Patriot Daily
American Family Association
American Flavor
American Freedom Fighters
American Journal Review
American Lookout
American News
American People Network
American Renaissance
American Today
American Updater
Americans Against the Tea Party (AATTP)
Aurora-News
A Voice for Men
Bare Naked Islam
BB4SP
Before It’s News
Big Blue Vision
Bipartisan Report
Black Genocide
Bluevision.news
Blue Vision Post
Breakingtop.world
CbsNews.com.co
Censored.News
Center for Immigration Studies (CIS)
CityWorldNews
Clarion Project
Clash Daily
Cnn.com.de
cnn-globalnews.com
Conservative Army
Conservative Daily Post
Conservative Fighters
Conservative Info Corner
Conservative Nation
Conservative News Flash
Conservapedia
Conservative 101
Conservative Post
Conservative Spirit
Conservative Tribune
C

In [11]:
data

{'': {'bias': 'C',
  'fact': 'VERY HIGH',
  'flag': None,
  'url': 'http://www.anthropocenemagazine.org/'},
 'Raw Progressive': {'bias': 'L',
  'fact': 'MIXED',
  'flag': None,
  'url': 'http://www.rawprogressive.com/'},
 'The Record (NJ)': {'bias': 'LC',
  'fact': 'HIGH',
  'flag': None,
  'url': 'http://www.northjersey.com/'},
 'The Automatic Earth': {'bias': 'LC',
  'fact': 'HIGH',
  'flag': None,
  'url': 'https://www.theautomaticearth.com/'},
 'Al Jazeera': {'bias': 'LC',
  'fact': 'HIGH',
  'flag': None,
  'url': 'http://www.aljazeera.com/'},
 'Lexington Institute': {'bias': 'RC',
  'fact': 'HIGH',
  'flag': None,
  'url': 'http://lexingtoninstitute.org/'},
 'TakePart': {'bias': 'L',
  'fact': 'HIGH',
  'flag': None,
  'url': 'http://www.takepart.com/'},
 'Broken World News': {'bias': None,
  'fact': None,
  'flag': 'S',
  'url': 'https://brokenworldnews.com/'},
 'Townhall': {'bias': 'R',
  'fact': 'HIGH',
  'flag': None,
  'url': 'http://townhall.com/'},
 'Left Action': {'bias':

In [15]:
with open('mbfc.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for key in data:
        writer.writerow([key, data[key]['bias'], data[key]['fact'], data[key]['flag'], data[key]['url']])